In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, recall_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import KFold


def test(file):
  # Wczytanie danych z pliku CSV
  data = pd.read_csv(file, sep=',', skipinitialspace=True)
  data['Class'] = data['Class'].str.strip()

  # Zakodowanie etykiet na liczby
  data['Class'] = data['Class'].replace({'positive': 1, 'negative': 0})

  # Podział danych na cechy i etykiety
  X = data.drop('Class', axis=1)
  y = data['Class']

  # Podział danych na zbiór treningowy i testowy
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=250)

  # Inicjalizacja walidacji krzyżowej (5-krotna)
  kf = KFold(n_splits=5, shuffle=True, random_state=250)

  clf = DecisionTreeClassifier()

  # Inicjalizacja SMOTE
  smote = SMOTE(random_state=250)

  # Inicjalizacja NearMiss
  near_miss = NearMiss()

  # Lista przechowująca accuracy dla każdego foldu
  accuracies = []

  # Lista przechowująca recall dla każdego foldu
  recalls = []

  # Iteracja po foldach
  for train_index, test_index in kf.split(X):
      # Podział danych na zbiór treningowy i testowy dla aktualnego foldu
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]

      # Podział danych treningowych na część SMOTE (90%) i NearMiss (10%)
      #X_train_smote, X_train_near_miss, y_train_smote, y_train_near_miss = train_test_split(X_train, y_train, test_size=ratio, random_state=250)

      # Zastosowanie SMOTE i NearMiss do danych treningowych
      X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)
      #X_train_resampled_near_miss, y_train_resampled_near_miss = near_miss.fit_resample(X_train_near_miss, y_train_near_miss)

      #Modyfikacja
      minority_class_size = y_train.value_counts().min()
      if minority_class_size > 1:
          smote_neighbors = min(minority_class_size - 1, 5)
          smote = SMOTE(random_state=250, k_neighbors=smote_neighbors)
          X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)
      else:
          X_train_resampled_smote, y_train_resampled_smote = X_train, y_train
      #if len(y_train_near_miss[y_train_near_miss == 1]) > 2:
          #near_miss = NearMiss(n_neighbors=3)
          #X_train_resampled_near_miss, y_train_resampled_near_miss = near_miss.fit_resample(X_train_near_miss, y_train_near_miss)
      #else:
          #X_train_resampled_near_miss, y_train_resampled_near_miss = X_train_near_miss, y_train_near_miss

      #Łączenie danych
      #X_train_resampled = pd.concat([X_train_resampled_smote, X_train_resampled_near_miss])
      #y_train_resampled = pd.concat([y_train_resampled_smote, y_train_resampled_near_miss])

      # Dopasowanie modelu do danych treningowych
      clf.fit(X_train_resampled_smote, y_train_resampled_smote)

      # Przewidywanie etykiet dla danych testowych
      y_pred = clf.predict(X_test)

      # Ocena dokładności modelu i dodanie wyniku do listy dokładności
      accuracy = accuracy_score(y_test, y_pred)
      accuracies.append(accuracy)

      recall = recall_score(y_test, y_pred)
      recalls.append(recall)

  #print(file)
  # Wyświetlenie dokładności dla każdego foldu
  #print("Accuracy dla poszczególnych foldów:", accuracies)

  # Obliczenie i wyświetlenie średniej accuracy z walidacji krzyżowej
  mean_accuracy = sum(accuracies) / len(accuracies)
  #print("Średnia accuracy:", mean_accuracy)

  # Wyświetlenie dokładności dla każdego foldu
  #print("Recall dla poszczególnych foldów:", recalls)

  # Obliczenie i wyświetlenie średniej recall z walidacji krzyżowej
  mean_recall = sum(recalls) / len(recalls)
  #print("Średnia recall:", mean_recall, "\n")

  print("\n", file)
  print("Accuracy:", mean_accuracy)
  print("Recall:", mean_recall)


files = ['/content/drive/MyDrive/Uczenie Maszynowe/Dane/ecoli-0_vs_1.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/glass0.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/glass1.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/haberman.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/vehicle1.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/vehicle2.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/wisconsin.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/yeast1.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/yeast3.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/yeast4.dat',
         '/content/drive/MyDrive/Uczenie Maszynowe/Dane/page-blocks0.dat'
         ]


for file in files:
    test(file)


 /content/drive/MyDrive/Uczenie Maszynowe/Dane/ecoli-0_vs_1.dat
Accuracy: 0.9636363636363636
Recall: 0.9615483870967741

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/glass0.dat
Accuracy: 0.7940199335548173
Recall: 0.7357789269553976

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/glass1.dat
Accuracy: 0.7428571428571428
Recall: 0.6574459527400704

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/haberman.dat
Accuracy: 0.620888418826018
Recall: 0.4557354925775979

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/vehicle1.dat
Accuracy: 0.7375565610859729
Recall: 0.5250610057807654

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/vehicle2.dat
Accuracy: 0.9656874347372085
Recall: 0.9268490374873354

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/wisconsin.dat
Accuracy: 0.9429261485616143
Recall: 0.9112831713803036

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/yeast1.dat
Accuracy: 0.6960778960778962
Recall: 0.5346022788164573

 /content/drive/MyDrive/Uczenie Maszynowe/Dane/yeast3.dat
Accurac